In [ ]:
using ITensors
using LaTeXStrings
using Plots
using ProgressMeter
using Statistics
using StatsBase

global const nsweeps::Int = 100
global const maxdim::Int = 1000
global const cutoff::Float64 = 1e-9
global const noise::Vector{Float64} = [1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 0.0]

global const observer::DMRGObserver{Float64} = DMRGObserver(energy_tol=1e-6)

global const rho::Float64 = (sqrt(5) - 1) / 2  # 1 / phi
global const rho2::Float64 = (3 - sqrt(5)) / 2;  # 1 / phi^2

ITensors.Strided.disable_threads()
ITensors.Strided.disable_threaded_mul()

In [ ]:
function build_HAos(ns::Int, A=20)
    N = 4 + 3*(ns - 1)
    
    os = OpSum()
    
    os += 2, "Z", 1, "Z", ns+1
    os += 2, "Z", N, "Z", 2*ns+1
    
    # source horizontal edge
    os += -1, "Z", 1, "Z", 2
    os += -1, "Z", 1, "Z", ns+2
    os += -2, "Z", 1
    # target horizontal edge
    os += -1, "Z", N, "Z", 2*ns
    os += -1, "Z", N, "Z", N-1
    os += -2, "Z", N

    # horizontal right edge
    os += +1, "Z", ns, "Z", 2*ns
    os += -1, "Z", ns, "Z", ns-1
    os += -1, "Z", ns, "Z", 2*ns+1
    os += +2, "Z", ns
    # horizontal left edge
    os += +1, "Z", 2*ns+2, "Z", ns+2
    os += -1, "Z", 2*ns+2, "Z", 2*ns+3
    os += -1, "Z", 2*ns+2, "Z", ns+1
    os += +2, "Z", 2*ns+2

    # vertical corner edges
    os += -1, "Z", ns+1, "Z", 2*ns+2
    os += -1, "Z", 2*ns+1, "Z", ns

    # lower horizontal internal edges
    for i in 2:ns-1
        os += +1, "Z", i, "Z", i+ns
        os += -1, "Z", i, "Z", i-1
        os += -1, "Z", i, "Z", i+ns+1
        os += -1, "Z", i, "Z", i+1
    end

    # vertical internal edges
    for i in ns+2:2*ns
        os += +1, "Z", i, "Z", i-ns
        os += +1, "Z", i, "Z", i+ns
        os += -1, "Z", i, "Z", i-ns-1
        os += -1, "Z", i, "Z", i+ns+1
        os += +4, "Z", i
    end

    # higher horizontal internal edges
    for i in 2*ns+3:N-1
        os += +1, "Z", i, "Z", i-ns
        os += -1, "Z", i, "Z", i-1
        os += -1, "Z", i, "Z", i-ns-1
        os += -1, "Z", i, "Z", i+1
    end
    
    os *= A / 4
    
    J = zeros(N, N)
    K = zeros(N)
    
    J[1, ns+1] += 2
    J[2*ns+1, N] += 2
    
    # source and tagert horizontal edges
    J[1, 2] += -1
    J[1, ns+2] += -1
    K[1] += -2
    
    J[2*ns, N] += -1
    J[N-1, N] += -1
    K[N] += -2

    # horizontal corner edges
    J[ns, 2*ns] += 1
    J[ns-1, ns] += -1
    J[ns, 2*ns+1] += -1
    K[ns] += 2
    
    J[ns+2, 2*ns+2] += 1
    J[2*ns+2, 2*ns+3] += -1
    J[ns+1, 2*ns+2] += -1
    K[2*ns+2] += 2
    
    # vertical corner edges    
    J[ns+1, 2*ns+2] += -1
    J[ns, 2*ns+1] += -1
    
    # lower horizontal internal edges
    for i in 2:ns-1
        J[i, i+ns] += 1
        J[i-1, i] += -1
        J[i, i+ns+1] += -1
        J[i, i+1] += -1
    end
    # vertical internal edges
    for i in ns+2:2*ns
        J[i-ns, i] += 1
        J[i, i+ns] += 1
        J[i-ns-1, i] += -1
        J[i, i+ns+1] += -1
        K[i] += 4
    end
    # higher horizontal internal edges
    for i in 2*ns+3:N-1
        J[i-ns, i] += 1
        J[i-1, i] += -1
        J[i-1-ns, i] += -1
        J[i, i+1] += -1
    end
    
    K = A/4 * K
    J = A/4 * J
    J = J .+ transpose(J)

    os = OpSum()
    for i in 1:N
        for j in i+1:N
            if J[i, j] != 0
                os .+= J[i, j], "Z", i, "Z", j
            end
        end
    end
    
    for i in 1:N
        if K[i] != 0
            os .+= K[i], "Z", i
        end
    end

    return os
end


function build_HBos(ns::Int, W, B=1)
    N = 4 + 3*(ns - 1)

    os = OpSum()
    for j in 1:N
        os += B/2*W[j], "Z", j
    end

    return os
end


function build_Hxos(ns::Int)
    N = 4 + 3*(ns - 1)
    
    os = OpSum()
    for j in 1:N
        os += -1, "X", j
    end

    return os
end;

In [ ]:
function compute_delta(sites::Vector{Index{Int64}}, N::Int, s::Float64, HA::MPO, HB::MPO, Hx::MPO; 
                       psi0_init=MPS()::MPS, psi1_init=MPS()::MPS)
    H = (1-s)*Hx + s*(HA + HB)
    
    if psi0_init.rlim == 0
        state = StatsBase.sample(["Up", "Dn"], N)
        psi0_init = MPS(sites, state)
    end
    
    E0, psi0 = dmrg(H, psi0_init; nsweeps, maxdim, cutoff, noise=noise,
                    eigsolve_krylovdim=4, outputlevel=0, observer=observer)
    
    if psi1_init.rlim == 0
        state = StatsBase.sample(["Up", "Dn"], N)
        psi1_init = MPS(sites, state)
    end
    
    E1, psi1 = dmrg(H, [psi0], psi1_init; nsweeps, maxdim, cutoff, noise=noise,
                    eigsolve_krylovdim=5, outputlevel=0, observer=observer, weight=2)
    
    return abs(E1-E0), psi0, psi1
end


function find_delta_min(sites::Vector{Index{Int64}}, N::Int, HA::MPO, HB::MPO, Hx::MPO; tol=1e-4::Float64)
    a = 0
    b = 0.5

    h = b - a
    c = a + rho2 * h
    d = a + rho * h
    
    # Required steps to achieve tolerance
    n = trunc(Int, ceil(log(tol / h) / log(rho)))

    yc, psi2, psi3 = compute_delta(sites, N, c, HA, HB, Hx)
    yd, psi0, psi1 = compute_delta(sites, N, d, HA, HB, Hx)
    
    for k in 1:n
        if yc < yd
            b = d
            d = c
            yd = yc
            h = rho * h
            c = a + rho2 * h

            yc, psi0, psi1 = compute_delta(sites, N, c, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
        else
            a = c
            c = d
            yc = yd
            h = rho * h
            d = a + rho * h

            yd, psi0, psi1 = compute_delta(sites, N, d, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
        end
    end
    
    if yc < yd
        sc = (a + d) / 2
    else
        sc = (b + c) / 2
    end
    
    dmin, psi0, psi1 = compute_delta(sites, N, sc, HA, HB, Hx, psi0_init=psi0, psi1_init=psi1)
    
    return sc, dmin
end;

In [ ]:
A = 20
B = 1

os_HA1 = OpSum()

os_HA1 +=  1, "Z", 1, "Z", 2
os_HA1 +=  1, "Z", 3, "Z", 4
os_HA1 += -1, "Z", 1, "Z", 3
os_HA1 += -1, "Z", 2, "Z", 4
os_HA1 *= A/2;

In [ ]:
A = 20
B = 1

ns = 5

N = 4 + 3*(ns - 1)
sites = siteinds("S=1/2", N)

if ns != 1
    os_HA = build_HAos(ns, A)
else
    os_HA = copy(os_HA1)
end
HA = MPO(os_HA, sites)

os_Hx = build_Hxos(ns)
Hx = MPO(os_Hx, sites)

rho = (sqrt(5) - 1) / 2 
s = rho * 0.5

os_HB = build_HBos(ns, rand(N), B)
HB = MPO(os_HB, sites);

In [ ]:
M = 1000
W = rand(N, M)
ScM = zeros(M)
DminM = zeros(M)
@showprogress for j in 1:M
    os_HB = build_HBos(ns, W[:, j], B)
    HB = MPO(os_HB, sites)

    ScM[j], DminM[j] = find_delta_min(sites, N, HA, HB, Hx)
end

In [ ]:
mean(ScM), std(ScM) / sqrt(M)

In [ ]:
mean(DminM), std(DminM) / sqrt(M)